In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
query = "SELECT * FROM FLIGHT_DATA"
flightsinfo = session.sql(query).to_pandas()

# Use pandas describe method for descriptive statistics
summary = flightsinfo.describe()

# Display the summary statistics
print(summary)

In [ ]:
# This function converts a given number
# of hours into a datetime.time object representing the corresponding time in hours and minutes.
# for example-->hours = 1230
# Output: 12:30:00
import datetime
def create_time_formatted(value):
# Convert float to integer by truncating the decimal part
    if isinstance(value, float):
        value = int(value)
    # Check if the value is an integer
    if isinstance(value, int):
    # Handle the special case for 2400
        if value == 2400:
            return datetime.time(0, 0)
    # Format the number as a 4-digit string (e.g., 900 becomes "0900")
        formatted_hours = "{0:04d}".format(value)
    # Extract the first two characters as hours and the last two as minutes
        hour_part = int(formatted_hours[:2])
        minute_part = int(formatted_hours[2:])
    # Return a time object using the extracted hours and minutes
        return datetime.time(hour_part, minute_part)

In [ ]:
# dropping all rows that contain NaN or missing value of selected columns
flightsinfo1 = flightsinfo.dropna(subset = ["TAIL_NUMBER",'DEPARTURE_TIME','DEPARTURE_DELAY','TAXI_OUT','WHEELS_OFF','ELAPSED_TIME','AIR_TIME','WHEELS_ON','TAXI_IN','ARRIVAL_TIME','ARRIVAL_DELAY'])

In [ ]:
flightsinfo1.isnull().sum()

In [ ]:
#Dropping all the non-contributing factors of delays as they are redundant ,we are only going to focus on the over al
flightsinfo2 = flightsinfo1.drop(['CANCELLATION_REASON','AIR_SYSTEM_DELAY','SECURITY_DELAY','AIRLINE_DELAY',
'LATE_AIRCRAFT_DELAY','WEATHER_DELAY'],axis = 1)

In [ ]:
flightsinfo2.isnull().sum()

In [ ]:
flightsinfo2.info()

In [ ]:
flightsinfo2[['WHEELS_OFF','DEPARTURE_TIME','SCHEDULED_DEPARTURE']]

In [ ]:
flightsinfo2['WHEELS_OFF'] = flightsinfo2['WHEELS_OFF'].apply(create_time_formatted)
flightsinfo2['Actual_Departure'] = flightsinfo1['DEPARTURE_TIME'].apply(create_time_formatted)
flightsinfo2['Scheduled_Departure'] = flightsinfo1['SCHEDULED_DEPARTURE'].apply(create_time_formatted)

In [ ]:
flightsinfo2[['Actual_Departure','Scheduled_Departure','WHEELS_OFF']]

In [ ]:
flightsinfo2['Scheduled_Arrival'] = flightsinfo1['SCHEDULED_ARRIVAL'].apply(create_time_formatted)
flightsinfo2['Actual_Arrival'] = flightsinfo2['ARRIVAL_TIME'].apply(create_time_formatted)

In [ ]:
flightsinfo2[['Scheduled_Arrival','Actual_Arrival']]

In [ ]:
# Function to convert HH:MM:SS to minutes
from datetime import datetime
def time_to_minutes(time_obj):
    return time_obj.hour * 60 + time_obj.minute

In [ ]:
flightsinfo2['SCHEDULED_DEPARTURE'] = flightsinfo2['Scheduled_Departure'].apply(time_to_minutes)
flightsinfo2['DEPARTURE_TIME'] = flightsinfo2['Actual_Departure'].apply(time_to_minutes)
flightsinfo2['WHEELS_OFF'] = flightsinfo2['WHEELS_OFF'].apply(time_to_minutes)

In [ ]:
flightsinfo2[['SCHEDULED_DEPARTURE','DEPARTURE_TIME','WHEELS_OFF']]

In [ ]:
flightsinfo2.columns

In [ ]:
flightsinfo2['DATE'] = pd.to_datetime(flightsinfo2[['YEAR','MONTH','DAY']])

In [ ]:
# Add a new column 'WEEK' which contains the day of the week from the 'DATE' column
flightsinfo2['WEEK'] = flightsinfo2['DATE'].dt.day_name()

In [ ]:
flightsinfo2['WEEK']

In [ ]:
#Create a TIME_OF_DAY column for further use in visualization purpose
# Function to categorize time of day
def categorize_time_of_day(minutes):
    if 240 <= minutes < 720:
        return 'Morning'
    elif 720 <= minutes < 1080:
        return 'Afternoon'
    elif 1080 <= minutes < 1260:
        return 'Evening'
    else:
        return 'Night'
# Apply the categorization function to create 'Time_of_Day'
flightsinfo2['TIME_OF_DAY'] = flightsinfo2['DEPARTURE_TIME'].apply(categorize_time_of_day)

In [ ]:
flightsinfo2['TIME_OF_DAY'].value_counts()

In [ ]:
query = "SELECT * FROM AIRPORT"
airport = session.sql(query).to_pandas()

# Use pandas' describe method for descriptive statistics
summary = airport.head()

# Display the summary statistics
print(summary)

In [ ]:
query = "SELECT * FROM AIRLINES"
airline = session.sql(query).to_pandas()

# Use pandas' describe method for descriptive statistics
summary = airline.head()

# Display the summary statistics
print(summary)

In [ ]:
flightsinfo2 = flightsinfo2.merge(airline, left_on='AIRLINE', right_on='IATA_CODE', how='inner')

In [ ]:
flightsinfo2.columns

In [ ]:
flightsinfo2 = flightsinfo2.drop(['IATA_CODE'], axis=1)
flightsinfo2 = flightsinfo2.rename(columns={"AIRLINE_x":"AIRLINE","AIRLINE_y":"AIRLINE_NAME"})

In [ ]:
flightsinfo2 = flightsinfo2.merge(airport, left_on='ORIGIN_AIRPORT', right_on='IATA_CODE', how='inner')
#The code merges the flightsinfo2 DataFrame with the airport DataFrame based on the 'ORIGIN_AIRPORT' and 'IATA_CODE'# keeping only rows that have matching values in both DataFrames

In [ ]:
flightsinfo2 = flightsinfo2.merge(airport, left_on='DESTINATION_AIRPORT', right_on='IATA_CODE', how='inner')
#The code merges the flightsinfo2 DataFrame with the airport DataFrame based on the 'DESTINATION_AIRPORT' and 'IATA_C# keeping only rows that have matching values in both DataFrames

In [ ]:
flightsinfo2.columns

In [ ]:
# Drop specified columns only if they exist in the DataFrame
columns_to_drop = ['LATITUDE_x', 'LONGITUDE_x', 'LATITUDE_y', 'LONGITUDE_y', 'index_x', 'index_y']
flightsinfo2 = flightsinfo2.drop(columns=[col for col in columns_to_drop if col in flightsinfo2.columns], axis=1)

# Display the remaining columns
print(flightsinfo2.columns)

In [ ]:
# Rename the columns in flightsinfo2
flightsinfo2 = flightsinfo2.rename(columns={
    'IATA_CODE_x': 'ORG_AIRPORT_CODE',
    'AIRPORT_x': 'ORG_AIRPORT_NAME',
    'CITY_x': 'ORGIN_CITY',
    'IATA_CODE_y': 'DEST_AIRPORT_CODE',
    'AIRPORT_y': 'DEST_AIRPORT_NAME',
    'CITY_y': 'DESTINATION_CITY' ##,'FLIGHT_COUNT_x' = '', 'FLIGHT_COUNT_y' =''
    
})

# Display the renamed columns
print(flightsinfo2.columns)

In [ ]:
# Save Snowpark DataFrame as a table

session.write_pandas(
    flightsinfo2, 
    "AIRLINE_ENRICHED", 
    auto_create_table=True)



In [ ]:
SELECT TOP 10 * FROM AIRLINE_ENRICHED

In [ ]:
DESC TABLE AIRLINE_ENRICHED;


In [ ]:
ALTER TABLE AIRLINE_ENRICHED ADD COLUMN converted_date TIMESTAMP_NTZ;


In [ ]:
UPDATE AIRLINE_ENRICHED
SET converted_date = TO_TIMESTAMP_NTZ(date / 1e9);


In [ ]:
ALTER TABLE AIRLINE_ENRICHED RENAME COLUMN converted_date TO DATE;
